# 如何为检索结果添加评分
[检索器](/docs/concepts/retrievers/)会返回[文档](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html)对象序列，默认情况下这些对象不包含关于检索过程的信息（例如与查询的相似度得分）。以下我们将演示如何将检索得分添加到文档的`.metadata`中：1. 摘自[向量存储检索器](/docs/how_to/vectorstore_retriever);2. 来自高阶LangChain检索器，例如[自查询检索器](/docs/how_to/self_query)或[多向量检索器](/docs/how_to/multi_vector)。
对于（1），我们将在对应的[向量存储](/docs/concepts/vectorstores/)外围实现一个简短的封装函数。对于（2），我们将更新对应类的一个方法。
## 创建向量存储
首先，我们用一些数据填充向量存储。我们将使用[PineconeVectorStore](https://python.langchain.com/api_reference/pinecone/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html)，但本指南兼容任何实现了`.similarity_search_with_score`方法的LangChain向量存储。

In [2]:
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]

vectorstore = PineconeVectorStore.from_documents(
    docs, index_name="sample", embedding=OpenAIEmbeddings()
)

## 检索器
为了从向量存储检索器中获取评分，我们将底层向量存储的`.similarity_search_with_score`方法封装在一个简短函数中，该函数会将评分打包到关联文档的元数据里。
我们为函数添加了一个 `@chain` 装饰器，用于创建一个可像典型检索器一样使用的 [Runnable](/docs/concepts/lcel)。

In [3]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*vectorstore.similarity_search_with_score(query))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score

    return docs

In [4]:
result = retriever.invoke("dinosaur")
result

(Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993.0, 'score': 0.84429127}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995.0, 'score': 0.792038262}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979.0, 'score': 0.751571238}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006.0, 'score': 0.747471571}))

请注意，上述文档的元数据中包含了检索步骤的相似度分数。

## SelfQueryRetriever
`SelfQueryRetriever` 将利用大语言模型（LLM）生成一种可能结构化的查询——例如，它能在常规的语义相似度驱动选择基础上，为检索构建过滤器。更多细节请参阅[本指南](/docs/how_to/self_query)。
`SelfQueryRetriever` 包含一个简短（1-2行）的方法 `_get_docs_with_query`，用于执行 `vectorstore` 的搜索。我们可以通过子类化 `SelfQueryRetriever` 并重写此方法来传递相似度分数。
首先，按照[操作指南](/docs/how_to/self_query)，我们需要建立一些用于筛选的元数据：

In [5]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm = ChatOpenAI(temperature=0)

我们随后重写 `_get_docs_with_query` 方法，以使用底层向量存储的 `similarity_search_with_score` 方法：

In [6]:
from typing import Any, Dict


class CustomSelfQueryRetriever(SelfQueryRetriever):
    def _get_docs_with_query(
        self, query: str, search_kwargs: Dict[str, Any]
    ) -> List[Document]:
        """Get docs, adding score information."""
        docs, scores = zip(
            *self.vectorstore.similarity_search_with_score(query, **search_kwargs)
        )
        for doc, score in zip(docs, scores):
            doc.metadata["score"] = score

        return docs

调用此检索器现在将在文档元数据中包含相似度分数。请注意，`SelfQueryRetriever` 底层的结构化查询能力仍然保留。

In [7]:
retriever = CustomSelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)


result = retriever.invoke("dinosaur movie with rating less than 8")
result

(Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993.0, 'score': 0.84429127}),)

## MultiVectorRetriever
`MultiVectorRetriever` 允许您将多个向量与单个文档关联。这在许多应用中非常有用。例如，我们可以对较大文档的小片段建立索引并在这些片段上执行检索，但在调用检索器时返回较大的“父”文档。[ParentDocumentRetriever](/docs/how_to/parent_document_retriever/) 作为 `MultiVectorRetriever` 的子类，提供了便捷方法来填充向量存储以支持此功能。更多应用详情请参阅本[操作指南](/docs/how_to/multi_vector/)。
为了通过该检索器传播相似度分数，我们可以再次子类化 `MultiVectorRetriever` 并重写一个方法。这次我们将重写 `_get_relevant_documents` 方法。
首先，我们准备一些模拟数据。我们生成模拟的“完整文档”并将其存储在文档存储中；这里我们将使用一个简单的[内存存储](https://python.langchain.com/api_reference/core/stores/langchain_core.stores.InMemoryBaseStore.html)。

In [8]:
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

# The storage layer for the parent documents
docstore = InMemoryStore()
fake_whole_documents = [
    ("fake_id_1", Document(page_content="fake whole document 1")),
    ("fake_id_2", Document(page_content="fake whole document 2")),
]
docstore.mset(fake_whole_documents)

接下来，我们将向向量存储中添加一些伪“子文档”。我们可以通过在其元数据中填充 `"doc_id"` 键来将这些子文档链接到父文档。

In [9]:
docs = [
    Document(
        page_content="A snippet from a larger document discussing cats.",
        metadata={"doc_id": "fake_id_1"},
    ),
    Document(
        page_content="A snippet from a larger document discussing discourse.",
        metadata={"doc_id": "fake_id_1"},
    ),
    Document(
        page_content="A snippet from a larger document discussing chocolate.",
        metadata={"doc_id": "fake_id_2"},
    ),
]

vectorstore.add_documents(docs)

['62a85353-41ff-4346-bff7-be6c8ec2ed89',
 '5d4a0e83-4cc5-40f1-bc73-ed9cbad0ee15',
 '8c1d9a56-120f-45e4-ba70-a19cd19a38f4']

为了传播评分，我们创建了`MultiVectorRetriever`的子类并重写了其`_get_relevant_documents`方法。这里我们将进行两处修改：
1. 我们将使用上述底层向量存储的`similarity_search_with_score`方法，在对应"子文档"的元数据中添加相似度分数；2. 我们将在检索到的父文档元数据中收录这些子文档的列表。此举能直观展示检索过程识别出的文本片段及其对应的相似度评分。

In [10]:
from collections import defaultdict

from langchain.retrievers import MultiVectorRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun


class CustomMultiVectorRetriever(MultiVectorRetriever):
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        """Get documents relevant to a query.
        Args:
            query: String to find relevant documents for
            run_manager: The callbacks handler to use
        Returns:
            List of relevant documents
        """
        results = self.vectorstore.similarity_search_with_score(
            query, **self.search_kwargs
        )

        # Map doc_ids to list of sub-documents, adding scores to metadata
        id_to_doc = defaultdict(list)
        for doc, score in results:
            doc_id = doc.metadata.get("doc_id")
            if doc_id:
                doc.metadata["score"] = score
                id_to_doc[doc_id].append(doc)

        # Fetch documents corresponding to doc_ids, retaining sub_docs in metadata
        docs = []
        for _id, sub_docs in id_to_doc.items():
            docstore_docs = self.docstore.mget([_id])
            if docstore_docs:
                if doc := docstore_docs[0]:
                    doc.metadata["sub_docs"] = sub_docs
                    docs.append(doc)

        return docs

调用该检索器时，我们可以发现它能够准确识别出父文档，并附上子文档中相关片段的相似度评分。

In [11]:
retriever = CustomMultiVectorRetriever(vectorstore=vectorstore, docstore=docstore)

retriever.invoke("cat")

[Document(page_content='fake whole document 1', metadata={'sub_docs': [Document(page_content='A snippet from a larger document discussing cats.', metadata={'doc_id': 'fake_id_1', 'score': 0.831276655})]})]